In [ ]:
 !pip install ppscore
 !pip install imblearn

  Created wheel for ppscore: filename=ppscore-1.1.1-py2.py3-none-any.whl size=12988 sha256=3f6c9ed6c6c77f3eff0603684b7fcba73d184c0f66179c37ffe360893d2b6339
  Stored in directory: /root/.cache/pip/wheels/3a/16/cb/4768ae3fd9f02af8cd6d880b9095281cc62bb1748fac67fdc4
Successfully built ppscore


In [ ]:
import ppscore as pps
import pandas as pd 
import numpy as np 

import random
import os

from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import accuracy_score, make_scorer, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
import collections

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


#Loading Dataset

In [ ]:
#uploading dataset
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
os.chdir("/content/gdrive/My Drive/459")

In [ ]:
os.chdir("/content/gdrive/My Drive/459")

In [ ]:
ls

 Final/              'Milestone 1 & 2'/   rf.csv        xgb.csv
'Final Report.gdoc'   rf2.gsheet          rf_temp.csv


#Reading Dataset

In [ ]:
data = pd.read_csv("/content/gdrive/My Drive/459/Final/milestone1.csv")
print(data.describe())

                 age       latitude  ...  Incidence_Rate  Case.Fatality_Ratio
count  456763.000000  456763.000000  ...   456763.000000        456763.000000
mean       43.277781      20.397071  ...      983.925039             2.967061
std        12.170669      17.757288  ...      950.046445             2.079119
min         0.000000     -53.166667  ...        0.029070             0.000000
25%        43.000000      13.083620  ...      279.548898             1.739222
50%        43.000000      19.200000  ...      744.467146             2.723007
75%        43.000000      28.614740  ...      983.839751             3.680701
max       106.000000      66.830900  ...     7938.444370            18.494473

[8 rows x 9 columns]


#Preprocessing

In [ ]:
data = data.drop('Last_Update',axis=1)
print(data.isnull().sum())
data = data.drop('sex',axis=1)

data['date_confirmation'] = pd.to_datetime(data['date_confirmation'], errors='coerce')
data = data.fillna(method='ffill')
data.date_confirmation = data.date_confirmation.apply(lambda x: int(x.strftime('%d%m%Y')))

pps_df = pps.predictors(data, y="outcome",cross_validation=10,random_seed=123)
pps_df = pps_df.sort_values(by="model_score",axis=0,ascending=False)
pps_df = pd.DataFrame(pps_df)
cols = pps_df[pps_df['ppscore']>0.1]['x'].values

age                         0
sex                    279349
province                    0
country                     0
latitude                    0
longitude                   0
date_confirmation         339
outcome                     0
Confirmed                   0
Deaths                      0
Recovered                   0
Active                      0
Combined_Key                0
Incidence_Rate              0
Case.Fatality_Ratio         0
dtype: int64


In [ ]:
outcome = data.outcome
data= data[cols]
data['outcome'] = outcome
data = data.drop('date_confirmation',axis=1)

le = preprocessing.LabelEncoder()
le = le.fit(data['outcome'])
data.outcome = le.transform(data.outcome.values)


for col in data.columns:
	if data[col].dtype == object:
		print(col)
		le = preprocessing.LabelEncoder()
		le = le.fit(data[col])
		data[col] = le.transform(data[col].values)
		data = data.drop(col,axis=1) 

province
Combined_Key
country


#Train-Test Split

In [ ]:
index_train = random.sample(range(len(data)),int(len(data)*.75))
train = data.iloc[index_train]
test = data.drop(index_train,axis=0)
print(len(train))
print(len(test))
y_train = train['outcome']
y_test = test['outcome']
print(y_test.value_counts())
print(y_train.value_counts())
X_train = train.drop('outcome',axis=1)
X_test = test.drop('outcome',axis=1)

342572
114191
1    49286
2    40066
3    23566
0     1273
Name: outcome, dtype: int64
1    147898
2    120066
3     70767
0      3841
Name: outcome, dtype: int64


#XGBoost Classifier

In [ ]:
xgb_model = XGBClassifier(learning_rate=0.01, booster='gbtree')

params = {  
            'min_child_weight': [1, 5],
            'max_depth': [3,6,10],
            'n_estimators': [20,50,100,200]
        }

scorers = {
    'f1_weighted' : make_scorer(f1_score, average='weighted'),
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score , average='weighted'),
    'accuracy': make_scorer(accuracy_score),
    'recall[deceased]' : make_scorer(recall_score,average=None,labels=[0])
}

# scorers = ['f1_weighted','precision_weighted','recall_weighted','accuracy']


skf = StratifiedKFold(n_splits=5, shuffle = True)

xgb_clf = GridSearchCV(xgb_model, 
                    param_grid = params,
                    scoring = scorers, 
                    n_jobs = -1, 
                    cv = skf.split(X_train, y_train),
                    refit = 'recall[deceased]')

xgb_clf.fit(X_train, y_train)

In [ ]:
xgb = pd.DataFrame(xgb_clf.cv_results_)
xgb = xgb[['param_max_depth',	'param_min_child_weight'	, 'param_n_estimators', 'mean_test_f1_weighted', 'mean_test_precision_weighted', 'mean_test_recall_weighted','mean_test_recall[deceased]', 'mean_test_accuracy', 'rank_test_accuracy']]
xgb = xgb.sort_values(by='rank_test_accuracy')
xgb.to_csv('xgb.csv',index=False)

In [ ]:
print(xgb_clf.best_params_)
print(xgb_clf.best_score_)

{'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 10}
0.03386723505816701


In [ ]:
xgb

,param_max_depth,param_min_child_weight,param_max_depth,param_n_estimators,mean_test_f1_weighted,mean_test_precision_weighted,mean_test_recall_weighted,mean_test_recall[deceased],mean_test_accuracy,rank_test_accuracy
18,10,0.5,10,200,0.761110,0.823142,0.808507,0.033867,0.808507,1
6,10,5,10,200,0.760904,0.822860,0.808359,0.033867,0.808359,2
4,10,0.5,10,20,0.757745,0.822373,0.806858,0.033079,0.806858,3
15,10,1,10,20,0.757734,0.822078,0.806838,0.033867,0.806838,4
0,10,5,10,100,0.757151,0.823013,0.806762,0.033604,0.806762,5
16,10,0.5,10,50,0.757325,0.822878,0.806753,0.033604,0.806753,6
12,10,1,10,50,0.757305,0.822576,0.806733,0.033604,0.806733,7
2,10,5,10,50,0.757213,0.822347,0.806636,0.033604,0.806636,8
10,6,0.5,6,200,0.756755,0.816330,0.805399,0.033342,0.805399,9
11,6,5,6,200,0.756681,0.816278,0.805355,0.032817,0.805355,10


In [ ]:
y_test_pred = xgb_clf.predict(X_test)
print("Accuracy Test {0:.2f}%".format(100*accuracy_score(y_test,y_test_pred)))
print(classification_report(y_test, y_test_pred))

Accuracy Test 81.11%
              precision    recall  f1-score   support

           0       0.67      0.03      0.05      1305
           1       0.70      0.98      0.82     49244
           2       1.00      1.00      1.00     40253
           3       0.80      0.17      0.29     23389

    accuracy                           0.81    114191
   macro avg       0.79      0.55      0.54    114191
weighted avg       0.83      0.81      0.76    114191



#Random Forest

In [ ]:
rf_model = RandomForestClassifier(bootstrap=True, warm_start=True, random_state=0)

param_dist = {  
            'n_estimators': [20,50,100,200],
            'max_features': ['auto', 'sqrt', 'log2'],
            'max_depth' : [None,4,6,8,10, 20, 30, 40, 50, 100],
            'criterion' :['gini', 'entropy']
        }

scorers = {
    'f1_weighted' : make_scorer(f1_score, average='weighted'),
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score , average='weighted'),
    'accuracy': make_scorer(accuracy_score),
    'recall[deceased]' : make_scorer(recall_score,average=None,labels=[0])
}


# scorers = ['f1_weighted','precision_weighted','recall_weighted','accuracy']


skf = StratifiedKFold(n_splits=5, shuffle = True)

clf_rf = RandomizedSearchCV(rf_model, 
                    param_dist, 
                    scoring = scorers, 
                    n_jobs = -1, 
                    cv = skf.split(X_train, y_train),
                    n_iter = 30,
                    refit = 'recall[deceased]')

clf_rf.fit(X_train, y_train)

In [ ]:
rf = pd.DataFrame(clf_rf.cv_results_)
rf = rf[['param_n_estimators',	'param_max_features',	'param_max_depth'	, 'param_criterion', 'mean_test_f1_weighted', 'mean_test_precision_weighted', 'mean_test_recall_weighted', 'mean_test_recall[deceased]','mean_test_accuracy', 'rank_test_accuracy']]
rf = rf.sort_values(by='rank_test_accuracy')
rf.to_csv('rf.csv',index=False)

In [ ]:
print(clf_rf.best_params_)
print(clf_rf.best_score_)

In [ ]:
y_test_pred = clf_rf.predict(X_test)
print(accuracy_score(y_test,y_test_pred))
print("Accuracy Test {0:.2f}%".format(100*accuracy_score(y_test,y_test_pred)))
print(classification_report(y_test, y_test_pred))

#KNN Classifier

In [ ]:
knn_model = KNeighborsClassifier()

param_dist = {  
            'n_neighbors': [3,4,5,8,10,15,20,25,30, 40, 50,100],
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']      
        }

scorers = {
    'f1_weighted' : make_scorer(f1_score, average='weighted'),
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score , average='weighted'),
    'accuracy': make_scorer(accuracy_score),
    'recall[deceased]' : make_scorer(recall_score,average=None,labels=[0])
}


# scorers = ['f1_weighted','precision_weighted','recall_weighted','accuracy']


skf = StratifiedKFold(n_splits=5, shuffle = True)

clf_knn = RandomizedSearchCV(knn_model, 
                    param_dist, 
                    scoring = scorers, 
                    n_jobs = -1, 
                    cv = skf.split(X_train, y_train),
                    n_iter = 30,
                    refit = 'recall[deceased]')

clf_knn.fit(X_train, y_train)

In [ ]:
knn = pd.DataFrame(clf_knn.cv_results_)
knn = knn[['param_n_neighbors','param_algorithm',	'param_weights'	, 'mean_test_f1_weighted', 'mean_test_precision_weighted', 'mean_test_recall_weighted','mean_test_recall[deceased]', 'mean_test_accuracy', 'rank_test_accuracy']]
knn = knn.sort_values(by='rank_test_accuracy')
knn.to_csv('knn.csv',index=False)
knn

In [ ]:
y_test_pred = clf_knn.predict(X_test)
print(accuracy_score(y_test,y_test_pred) )
print("Accuracy Test {0:.2f}%".format(100*accuracy_score(y_test,y_test_pred)))
print(classification_report(y_test, y_test_pred))

In [ ]:
print(clf_knn.best_params_)
print(clf_knn.best_score_)